# Data

We will load the data from a specific path.

In [ ]:
import pandas as pd

dataset = pd.read_csv("../data/1a_3d_200s_0.5ir_4321_2025-08-10.csv")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(
    x=dataset["x0"], y=dataset["x2"], alpha=0.6, hue=dataset["y"], palette="deep"
)
plt.show()

In [ ]:
X_sample = dataset.drop(columns=["y"])
y_sample = dataset["y"]

## Preprocessing step

Based on feature correlation.

In [ ]:
from kernel_trainer.preprocess import Preprocessor

# Number of dimensions
num_dimensions = 2
prep = Preprocessor(num_dimensions, mode="pca", scale=True)
X_reduced = prep.fit_transform(X_sample)

In [ ]:
from sklearn.model_selection import train_test_split

# Split train and test
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y_sample, test_size=0.20, random_state=42
)

# Kernel trainer

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)


In [ ]:
from kernel_trainer import kernel_generator

n_pop = 5
n_gen = 2
chain = 6
mutation_pb = 0.6
cx_prob = 0.3

pop_final, log, _ = kernel_generator(
    X_train,
    y_train,
    num_pop=n_pop,
    ngen=n_gen,
    cxpb=cx_prob,
    mutpb=mutation_pb,
    chain_size=chain,
    penalize_complexity=False,
)

2026-02-02 13:00:36.215 | DEBUG    | kernel_trainer.metrics:qiskit_target_alignment:69 - Kernel evaluated at 669206 (MEM: 300.7)
2026-02-02 13:00:36.371 | DEBUG    | kernel_trainer.kernels.quantum:evaluation_function:573 - Finishing 669206 (MEM: 287.96 MB)
2026-02-02 13:00:36.372 | DEBUG    | kernel_trainer.kernels.quantum:evaluation_function:575 - Proxy calculation on 669206 took 36.053521394729614 (proc. time 44.852553235)
2026-02-02 13:00:36.374 | DEBUG    | kernel_trainer.kernels.quantum:evaluation_function:545 - Creating individual in process 669206 (MEM: 287.96 MB)
2026-02-02 13:00:36.376 | DEBUG    | kernel_trainer.kernels.quantum:evaluation_function:556 - Going for the proxy metric 669206 (MEM: 287.96 MB)
2026-02-02 13:01:17.490 | DEBUG    | kernel_trainer.metrics:qiskit_target_alignment:69 - Kernel evaluated at 669206 (MEM: 316.54)
2026-02-02 13:01:17.592 | DEBUG    | kernel_trainer.kernels.quantum:evaluation_function:573 - Finishing 669206 (MEM: 305.77 MB)
2026-02-02 13:01:17

In [ ]:
best = pop_final[0]
best

In [ ]:
from qiskit import QuantumCircuit
from kernel_trainer.kernels.quantum import ind_to_qiskit_kernel

qc = QuantumCircuit(X_train.shape[1])

kernel_auto = ind_to_qiskit_kernel(best, qc)
kernel_auto.feature_map.decompose().draw("mpl", fold=-1)

In [ ]:
kernel_auto.feature_map.depth()

In [ ]:
kernel_auto.feature_map.num_nonlocal_gates()

In [ ]:
ops = kernel_auto.feature_map.count_ops()
sum([n for _, n in ops.items()])

In [ ]:
import pennylane as qml
from kernel_trainer.kernels.quantum import ind_to_pennylane_kernel

qml.drawer.use_style("sketch")

dev = qml.device("lightning.qubit", wires=num_dimensions)

kernel_auto = ind_to_pennylane_kernel(pop_final[0], dev)
print(qml.draw_mpl(kernel_auto)(X_train[0], X_train[0]))

In [ ]:
specs = qml.specs(qnode=kernel_auto)(X_train[0], X_train[0])
specs["resources"]

In [ ]:
specs["resources"].depth

In [ ]:
specs["resources"].gate_types["PauliRot"]

Now running with Pennylane backend.

In [ ]:
backend = "pennylane"

pop_final, log, _ = kernel_generator(
    X_train,
    y_train,
    backend=backend,
    num_pop=n_pop,
    ngen=n_gen,
    cxpb=cx_prob,
    mutpb=mutation_pb,
    chain_size=chain,
)

In [ ]:
kernel_auto = ind_to_pennylane_kernel(pop_final[0], dev)
print(qml.draw_mpl(kernel_auto)(X_train[0], X_train[0]))